In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import gensim

E:\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# idee

NLP:
    - pos tagging per contare numero di aggettivi 
    - qualcosa per vedere se aggettivi sono positivi o negativi (direttamente sentiment analysis)?
    - rimuovere gli @ vs tenere solo i più frequenti 
    - rimuovere i link
    - pulire gli hashtag? 
    - word2vec distance per avere simili tweet assieme nello spazio vettoriale?
    - ridurre parole presenti? 
    - tfidf?
    
Processo:
    - clustering interno all'anno
    - clustering generale 
    - clustering con/senza sentiment meaning

In [2]:
data = pd.read_json("dataWithTags.json")
data['created_at'] = pd.to_datetime(data.created_at, format="%Y-%m-%d %H:%M:%S")
data.sort_values(by=['created_at'], inplace=True, ascending=True)
data.reset_index(inplace=True,drop=True)
data

,created_at,favorite_count,id_str,in_reply_to_user_id_str,is_retweet,retweet_count,source,text,modded_text,@johnboehner,...,@trumpdoral,@dianejackson9,@toure,@floydmayweather,@10ajmccarron,@rspbscotland,@newyorkgop,@mattginellagc,@foxandfriends,@cnn
0,2009-05-04 18:54:25,202,1698308935,NaN,False,253,Twitter Web Client,Be sure to tune in and watch Donald Trump on L...,be sure to tune in and watch donald trump on l...,0,...,0,0,0,0,0,0,0,0,0,0
1,2009-05-05 01:00:10,3,1701461182,NaN,False,2,Twitter Web Client,Donald Trump will be appearing on The View tom...,donald trump will be appearing on the view tom...,0,...,0,0,0,0,0,0,0,0,0,0
2,2009-05-08 13:38:08,2,1737479987,NaN,False,3,Twitter Web Client,Donald Trump reads Top Ten Financial Tips on L...,donald trump reads top ten financial tips on l...,0,...,0,0,0,0,0,0,0,0,0,0
3,2009-05-08 20:40:15,27,1741160716,NaN,False,8,Twitter Web Client,New Blog Post: Celebrity Apprentice Finale and...,new blog post celebrity apprentice finale and ...,0,...,0,0,0,0,0,0,0,0,0,0
4,2009-05-12 14:07:28,1950,1773561338,NaN,False,1421,Twitter Web Client,"""My persona will never be that of a wallflower...",my persona will never be that of a wallflower ...,0,...,0,0,0,0,0,0,0,0,0,0
5,2009-05-12 19:21:55,13,1776419923,NaN,False,10,Twitter Web Client,"Miss USA Tara Conner will not be fired - ""I've...",miss usa tara conner will not be fired ive al...,0,...,0,0,0,0,0,0,0,0,0,0
6,2009-05-13 17:38:28,10,1786560616,NaN,False,11,Twitter Web Client,Listen to an interview with Donald Trump discu...,listen to an interview with donald trump discu...,0,...,0,0,0,0,0,0,0,0,0,0
7,2009-05-14 16:30:40,6,1796477499,NaN,False,3,Twitter Web Client,"""Strive for wholeness and keep your sense of w...",strive for wholeness and keep your sense of wo...,0,...,0,0,0,0,0,0,0,0,0,0
8,2009-05-15 14:13:13,2,1806258917,NaN,False,1,Twitter Web Client,"Enter the ""Think Like A Champion"" signed book ...",enter the think like a champion signed book an...,0,...,0,0,0,0,0,0,0,0,0,0
9,2009-05-16 22:22:45,5,1820624395,NaN,False,3,Twitter Web Client,"""When the achiever achieves, it's not a platea...",when the achiever achieves its not a plateau i...,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
data.drop(["in_reply_to_user_id_str","id_str","source"], inplace=True,axis=1) 

# Aggiungi dati per il tempo

In [ ]:
#exploit time
data['month'] = [d.month for d in data['created_at']]
data["year"] = [d.year for d in data["created_at"]]
data["hour"] = [d.hour for d in data["created_at"]]
data["week_year"] = [d.weekofyear for d in data["created_at"]]
data["date"] = [d.date() for d in data["created_at"]]
temp = data.groupby(["year"])["year"].count()
plt.plot(temp)

annuncio campagna elettorale giugno 2015

In [ ]:
temp = pd.DataFrame({'count' : data.groupby( [ "year", "month"] ).size()}).reset_index()
#temp
#plotta nel corso dei mesi il numero di tweet per anni, devi prendere i dati per ogni anno e farli di lunghezza uguale come array e poi plotti 

l = []
for year in temp["year"].unique().tolist():
    l1 = [0] * 12
    t = temp[temp["year"] == year]
    for idx,row in t.iterrows():
        l1[row["month"]-1] = row["count"]
    l.append(l1)

leg = temp["year"].unique().tolist()
fig, ax = plt.subplots(figsize = (15, 10))
months = [x for x in range(0,12)]
for i in range(0,len(l)):
    plt.plot(months, l[i],label = leg[i])
plt.legend()

In [ ]:
data.drop("created_at", inplace=True,axis=1)
data.drop("is_retweet", inplace=True,axis=1)
data

# trasforma in label i valori delle date

In [ ]:
from sklearn.preprocessing import LabelEncoder

#trasforma la data in una label
labeller = LabelEncoder()
data["date"] = labeller.fit_transform(data["date"])

In [ ]:
data

# Stemming and removing stopwords 


In [ ]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()
stemmedTweets = []

for idx, row in data.iterrows():
    test = row["modded_text"].split()
    desc = ""
    for el in test:
        desc = desc + " " + stemmer.stem(el) + " "
    stemmedTweets.append(desc)
temp = pd.Series(stemmedTweets)
data["stemmed"] = temp

In [ ]:
#remove stopwords
import nltk
from nltk.corpus import stopwords

stop = stopwords.words('english')

data['stemmed'] = data['stemmed'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
data.head()

In [ ]:
stemmed_words = pd.Series(' '.join(data['stemmed']).split()).value_counts()
print(len(stemmed_words))
stemmed_words

In [ ]:
stemmed_words1 = stemmed_words[stemmed_words > 3]
stemmed_words1

In [ ]:
data['stemmed_counted'] = data['stemmed'].apply(lambda x: " ".join(x for x in x.split() if x in stemmed_words1))
data["stemmed_counted"]

# Word2vec ? 

In [ ]:
from gensim.models import Word2Vec
from time import time  # To time our operations
from gensim.test.utils import common_texts, get_tmpfile

postElection = data[data["year"] >= 2015]

corpus = []
for idx,row in postElection.iterrows():
    c = gensim.utils.simple_preprocess(row["stemmed_counted"])  #apply a bit of preprocessing done automatically
    corpus.append(c)

w2v_model = Word2Vec(min_count=4, window=2,size=300,alpha=0.03,negative=20)


In [ ]:
t = time()

w2v_model.build_vocab(corpus, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))
len(w2v_model.wv.vocab)

In [ ]:
w2v_model.wv.vocab

In [ ]:
t = time()

w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))


In [ ]:
w2v_model.wv.most_similar(negative=["hillaryclinton"])


 Capisci come applicare a tutti i dati che ti interessa kmeans

# ROBA

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import scipy.sparse as sp
from scipy.sparse import hstack


data1 = data.drop(["text","modded_text"],axis=1)
data1 = data[data["year"] == 2018]
data2 = data1[["stemmed_counted","date"]]

vectorizer = TfidfVectorizer(stop_words='english')

X = vectorizer.fit_transform(data2["stemmed_counted"])

#for i, col in enumerate(vectorizer.get_feature_names()):
#    data2[col] = pd.SparseSeries(X[:, i].toarray().ravel(), fill_value=0)


df1 = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())   
df1
#data2


true_k = 20
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)


In [ ]:
print(model.inertia_)
#con l'ultima maniera è più bassa di 1000 rispetto a fede, viene tipo 2400 alei 3800
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print
#3170

In [ ]:
from sklearn.metrics import silhouette_score
silhouette_score(X, model.labels_)


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import scipy.sparse as sp
from scipy.sparse import hstack
import numpy as np

data1 = data[data["year"] == 2018]

vect = TfidfVectorizer()
vect.fit(data1['stemmed_counted'])
print("vocabulary size: {}".format(len(vect.vocabulary_)))
#X_train_vectored = vect.transform(data1['stemmed_counted'])
#X_train_vectored_fix=X_train_vectored
#X_train_vectored = X_train_vectored_fix

data3 = X_train_vectored
#features = ["date",'favorite_count', 'retweet_count']
#for feature in features:
#    data3 = hstack((X_train_vectored,np.array(data1[feature])[:,None]))

true_k = 30
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(data3)